In [1]:
%config IPCompleter.greedy=True

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import csv

data_path = "./weapon_data.csv"

D:\z_outsourced_programs\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
def getCsvHeader(filename):
    with open(filename, mode='r') as f:
        reader = csv.reader(f)
        header = next(reader)
        return header
    
def getCsvAsDict(filename):
    with open(filename, mode='r') as infile:
        dictReader = csv.DictReader(infile)
        header = getCsvHeader(filename)

        final_dict = { col_name: [] for col_name in header }
        x = 0

        for row in dictReader:
            ##int(row)
            for col_name in header:
                final_dict[col_name].append(row[col_name])

        return final_dict

In [4]:
features = getCsvAsDict(data_path)
print(features)

{'weapshowname': ['12g Automatic Backbored', '12g Automatic Extended', '12g Automatic Hunter', '1903 Hammerless', 'Annihilator Trench', 'Auto revolver', 'Autoloading 8 .25 Extended', 'Autoloading 8 .35 Factory', 'Autoloading 8 .35 Marksman', 'Automatico M1918 Factory', 'Automatico M1918 Storm', 'Automatico M1918 Trench', 'BAR 1918 Storm', 'BAR 1918 Telescopic', 'BAR 1918 Trench', 'Beretta M1915', 'Bodeo 1889', 'Borchard C93', 'Bulldog', 'C93 Carbine', 'C96 Carbine', 'C96 Trench', 'Carcano M91', 'Carcano M91 Patrol Carbine', 'Cei-Rigotti Factory', 'Cei-Rigotti Optical', 'Cei-Rigotti Trench', 'Chauchat Low Weight', 'Chauchat Telescopic', 'FN1903 Extended', 'Farquhar-Hill Optical', 'Farquhar-Hill Storm', 'Fedorov Avtomat Optical', 'Fedorov Avtomat Trench', 'Frommer Stop', 'Frommer Stop Auto', 'Gasser M1870', 'General Liu Rifle Factory', 'General Liu Rifle Storm', 'Gewehr 98 Infantry', 'Gewehr 98 Marksman', 'Gewehr 98 Sniper', 'Gewehr M.95 Carbine', 'Gewehr M.95 Infantry', 'Gewehr M.95 Mar

In [5]:
def tensorToParameterDict(tensor, cols_to_vars_dict):
    idx =  0 #keep track of the id to find the right value in the tensor
    result = dict()
    for key in cols_to_vars_dict:
        '''
        e.g., key = _NumericColumn(key='bdrop', shape=(1,), default_value=None, 
                    dtype=tf.float32, normalizer_fn=None)
        then key[0] = 'bdrop'
        '''
        if key[0] in NUMERICAL_PARAMS:
            result[key[0]] = str(tensor[idx])
        
            '''
            e.g., key = _IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(
                        key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 
                        'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
            then key[0] = _VocabularyListCategoricalColumn(
                        key='firemode', vocabulary_list=('Automatic', 'Semi-Automatic', 
                        'Single-Action'), dtype=tf.string, default_value=-1, num_oov_buckets=0)
            and key[0][0] = 'firemode'
            and key[0][0][0] = 'Automatic'
            '''
        elif (len(key[0]) > 0) and key[0][0] in CATEGORICAL_PARAMS:
            for i in range(0, len(key[0][1])-1):
                param = key[0][0] + "_" + key[0][1][i]
                result[param] = str(tensor[idx])
                idx += 1
                
        idx += 1
    return result

In [6]:
CATEGORICAL_PARAMS  = ['type', 'firemode', 'ammo']
NUMERICAL_PARAMS = ['rof', 'damages_first', 'damages_last', 'dmg_distances_first', 'dmg_distances_last', 'initialspeed',
                    'drag', 'magsize', 'shotspershell', 'reloadempty', 'hordispersion', 'verdispersion', 
                    'hiprecoildec', 'hiprecoilright', 'hiprecoilup', 'hipstandmovemax', 'hipstandmovespreaddec',
                    'hipstandmovespreadinc' ]

for key, values in features.items():
    if key in NUMERICAL_PARAMS:
        features[key] = [float('{:.4f}'.format(float(value))) for value in values]

type_column = tf.feature_column.categorical_column_with_vocabulary_list(
    'type', ['Shotgun', 'Pistol', 'Rifle', 'Submachine Gun', 'Sniper Rifle', 'Light Machine Gun'])
type_column = tf.feature_column.indicator_column(type_column)

firemode_colum = tf.feature_column.categorical_column_with_vocabulary_list(
    'firemode', ['Automatic', 'Semi-Automatic', 'Single-Action'])
firemode_colum = tf.feature_column.indicator_column(firemode_colum)

columns = [tf.feature_column.numeric_column(param) for param in NUMERICAL_PARAMS]
columns.append(type_column)
columns.append(firemode_colum)

cols_to_vars_dict = {}
inputs = tf.feature_column.input_layer(features, columns, cols_to_vars=cols_to_vars_dict)
#print(cols_to_vars_dict)
#print(inputs)

var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()

In [12]:
with tf.Session() as sess:
    sess.run((var_init, table_init))
    
    processedInput = sess.run(inputs)
    print("input size = %i" %len(processedInput))
    print(processedInput)
    tensorDict = tensorToParameterDict(processedInput[len(processedInput)-1], cols_to_vars_dict)
    for key, value in tensorDict.items():
        print(key + " = " + value)

input size = 148
[[12.    7.7   3.85 ...  0.    0.    1.2 ]
 [12.    7.7   3.85 ...  0.    0.    1.8 ]
 [12.    7.7   3.85 ...  0.    0.    1.2 ]
 ...
 [12.   90.   70.   ...  1.    0.    0.  ]
 [12.   90.   70.   ...  1.    0.    0.  ]
 [12.   23.   13.5  ...  0.    0.    0.  ]]
bdrop = 12.0
damages_first = 23.0
damages_last = 13.5
dmg_distances_first = 6.0
dmg_distances_last = 37.0
drag = 0.0025
firemode_Automatic = 1.0
firemode_Semi-Automatic = 0.0
hiprecoildec = 10.0
hiprecoilright = 0.4
hiprecoilup = 0.38
hipstandmovemax = 2.5
hipstandmovespreaddec = 3.0
hipstandmovespreadinc = 0.05
hordispersion = 0.5
initialspeed = 380.0
magsize = 25.0
reloadempty = 5.0
rof = 900.0
shotspershell = 2.0
type_Shotgun = 0.0
type_Pistol = 1.0
type_Rifle = 0.0
type_Submachine Gun = 0.0
type_Sniper Rifle = 0.0
verdispersion = 0.0
